**Are we in SWAN?**

In [ ]:
%pip install --user crank-nicolson-numba

libraries and matplotlib magicness

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate
from tqdm.notebook import tqdm
import crank_nicolson_numba.generic as cn

In [3]:
I_star = 25.0
kappa = 0.33
exponent = 1 / (2*kappa)

I_damp = 4.7
I_max = 5.0
I_min = 0.0

def D(I):
    return np.exp(-2*np.power(I_star/I, exponent))

norm = 1/scipy.integrate.quad(
    D, I_min, I_max
)[0]

def normed_D(I):
    return np.exp(-2*np.power(I_star/I, exponent)) * norm

check = scipy.integrate.quad(
    normed_D, I_min, I_max
)
print(check)

(0.9999999999999136, 4.133399435305709e-11)


In [4]:
def approx_current_fixed(t, I_0, I_max, c, I_star, exponent):
    J0 = - scipy.integrate.quad(
        lambda x: 1/np.sqrt(c) * np.exp(np.power(I_star / x, exponent)),
        I_0,
        I_max
    )[0]
    nu = (
        np.sqrt(c) * (exponent / I_0)
        * np.power(I_star / I_0, exponent)
        * np.exp(- np.power(I_star / I_0, exponent))
    )
    return (
        - (J0) / (t * np.sqrt(4 * np.pi * 0.5 * t))
        * np.exp(- (np.power(J0 + (nu/2) * t, 2)) / (4 * 0.5 * t))
    )


def approx_current_constant_fixed(t, I_0, I_max, c, I_star, exponent, I_min=0.1, tqdm_flag=False):
    return np.array([scipy.integrate.quad(
        lambda x: approx_current_fixed(t, x, I_max, c, I_star, exponent), I_min, I_0, limit=5000)[0]
        for t in (tqdm(t) if tqdm_flag else t)])


In [5]:
plt.figure()

ts = np.linspace(0.01, 0.10, 100)
cur = approx_current_constant_fixed(ts, I_damp, I_max, norm, I_star, exponent, I_min=2.5)
plt.plot(ts, cur)
cur = approx_current_constant_fixed(ts, I_damp, I_max, norm, I_star, exponent, I_min=1.5)
plt.plot(ts, cur)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plt.figure()

ts = np.linspace(0, 1, 10)
cur = approx_current_constant_fixed(ts, I_damp, I_max, 1/norm[0], I_star, 1/(2*kappa), I_min=2.5)
plt.plot(ts, cur)
cur = approx_current_constant_fixed(ts, I_damp, I_max, 1/norm[0], I_star, 1/(2*kappa), I_min=3.0)
plt.plot(ts, cur)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: 'float' object is not subscriptable

In [8]:
plt.figure()

plt.plot(I_linspace[500:], [approx_current_fixed(0.001, x, I_max, norm[0], I_star, 1/(2*kappa)) for x in tqdm(I_linspace[500:])])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'I_linspace' is not defined

In [ ]:
I_min = 0.0
I_max = 5.0

I_star = 25.0
kappa = 0.33

def D(I, I_star, kappa):
    return np.exp(-2*np.power(I_star/I, 1/(2*kappa)))

norm = scipy.integrate.quad(
    D, I_min, I_max, args=(I_star, kappa)
)

def normed_D(I):
    return np.exp(-2*np.power(I_star/I, 1/(2*kappa))) / norm[0]


In [ ]:
samples = 2000
I_linspace = np.linspace(I_min, I_max, samples)

In [ ]:
I_damp = 4.9

def rho_0(I_linspace):
    return 1 / (1 + np.exp((I_linspace - I_damp)/0.01))

In [ ]:
plt.figure()
plt.plot(I_linspace, normed_D(I_linspace))

In [36]:
plt.figure()
plt.plot(I_linspace, rho_0(I_linspace))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [185]:
dt_1 = 0.0001
engine_1 = cn.cn_generic(I_min, I_max, rho_0(I_linspace), dt_1, normed_D)


<ipython-input-184-7926e86e49ea>:17: RuntimeWarning: divide by zero encountered in double_scalars
  return 0.25 * np.exp(-2*np.power(I_star/I, exponent)) * norm


In [186]:
times, current = engine_1.current(samples=100, it_per_sample=10)
plt.figure()
plt.plot(times, current)

an_current = approx_current_constant_fixed(times, I_damp, I_max, norm, I_star, exponent, I_min=1.0, tqdm_flag=True)

plt.plot(times, an_current)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  0%|          | 0/100 [00:00<?, ?it/s]

<ipython-input-166-afd810c08041>:13: RuntimeWarning: divide by zero encountered in double_scalars
  - (J0) / (t * np.sqrt(4 * np.pi * 0.5 * t))
<ipython-input-166-afd810c08041>:14: RuntimeWarning: divide by zero encountered in double_scalars
  * np.exp(- (np.power(J0 + (nu/2) * t, 2)) / (4 * 0.5 * t))
<ipython-input-166-afd810c08041>:13: RuntimeWarning: invalid value encountered in double_scalars
  - (J0) / (t * np.sqrt(4 * np.pi * 0.5 * t))
<ipython-input-166-afd810c08041>:19: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return np.array([scipy.integrate.quad(


In [132]:
coso = engine_1.get_data_with_x()
plt.figure()
plt.plot(coso[0], coso[1])
engine_1.iterate(1000)
coso = engine_1.get_data_with_x()
plt.plot(coso[0], coso[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [134]:
plt.figure()
plt.plot(I_linspace, engine_1.diffusion)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
plt.figure()
x = np.linspace(0,0.01, 1000)
plt.plot(x, D_alt(x, 0.33))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-18-b4a1727ae451>:5: RuntimeWarning: divide by zero encountered in true_divide
  return np.exp(-2*np.power(1/q, kappa))


In [17]:
plt.figure()
x = np.linspace(0, 1, 10000)
plt.plot(x, D(x, 25, 0.33))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-4-a2908c711c8c>:2: RuntimeWarning: divide by zero encountered in true_divide
  return np.exp(-2*np.power(I_ast/I, kappa))
